In [0]:
%python
service_credential = dbutils.secrets.get(
    scope="azure-sp",
    key="client-secret"
)

adls_configs = {
    "fs.azure.account.auth.type.deproj1adls.dfs.core.windows.net": "OAuth",
    "fs.azure.account.oauth.provider.type.deproj1adls.dfs.core.windows.net": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id.deproj1adls.dfs.core.windows.net": "aca11b08-9a74-41fe-9d44-09d033504c53",
    "fs.azure.account.oauth2.client.secret.deproj1adls.dfs.core.windows.net": service_credential,
    "fs.azure.account.oauth2.client.endpoint.deproj1adls.dfs.core.windows.net": "https://login.microsoftonline.com/df6de6ef-8435-42ec-ad06-b4ca441bfdf4/oauth2/token"
}

df = spark.read.format("delta").options(
    **adls_configs
).load(
    "abfss://gold@deproj1adls.dfs.core.windows.net/rebrickable_api/Minifigs/dim_minifigs"
)


In [0]:
from pyspark.sql.functions import *

fact_df = df.agg(
    current_date().alias("snapshot_date"),
    count("*").alias("total_minifigs"),
    round(avg("num_parts"),2).alias("avg_parts_per_minifig"),
    min("num_parts").alias("min_parts"),
    max("num_parts").alias("max_parts"),
    sum("is_superhero_flag").alias("superhero_minifigs_count")
)


In [0]:
fact_df.write\
  .format("delta")\
  .options(**adls_configs)\
  .save("abfss://gold@deproj1adls.dfs.core.windows.net/rebrickable_api/Minifigs/fact_minifigs_stats")
    